In [4]:
import numpy as np
import pandas as pd
import warnings
import os
from tqdm import tqdm
from sklearn import preprocessing, metrics
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GroupKFold, KFold, StratifiedKFold
import gc
import lightgbm as lgb
import matplotlib.pyplot as plt
import seaborn as sns
from joblib import Parallel, delayed
from sklearn.utils import shuffle

%matplotlib inline

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

warnings.filterwarnings('ignore')

In [5]:
seed = 2020

In [6]:
df_feature1 = pd.read_pickle('./temp/part1_speicial_feature.plk')
df_feature2 = pd.read_pickle('./temp/part2_feature.plk')

In [7]:
#df_feature2=df_feature2.drop(['id'],axis=1)
df_feature2.head()

,id,courier_id,wave_index,tracking_id,courier_wave_start_lng,courier_wave_start_lat,action_type,expect_time,date,type,target,group,current_time,last_tracking_id,last_action_type,source_lng,source_lat,target_lng,target_lat,grid_distance,TA,aoi_id,shop_id,promise_deliver_time,estimate_pick_time,assigned_time,speed,max_load,weather_grade,level,target_MHD,promise_dif,estimate_dif,Restime_dif,assign_dif
0,150310,10290949,5,2100075633932043269,121.411024,39.182940,0,1582538861,20200224,train,1.0,33303,1582538560,2100075631583036554,0,121.416654,39.203351,121.417974,39.201901,958.0,105637,14639,7765,1582540148,1582537688,1582537174,0.717700,0.544444,0.100000,1.1,0.131356,1588,-872,2460,-1386
1,32147,10298963,1,2100074804874806590,121.531926,39.250766,0,1581051114,20200207,train,1.0,7443,1581050821,2100074805553628748,1,121.526993,39.260214,121.527484,39.261721,549.0,23300,24807,1794,1581052142,1581050283,1581049640,0.554962,0.655556,1.100000,1.1,0.122617,1321,-538,1859,-1181
2,189309,117054417,0,2100075865005852254,121.449011,39.094955,1,1582857411,20200228,train,1.0,41544,1582857111,2100075865747752238,0,121.437904,39.097161,121.439220,39.100178,616.0,124460,6151,8358,1582863668,1582855928,1582855297,0.686344,0.655556,0.100000,1.1,0.149050,6557,-1183,7740,-1814
3,17842,117026449,2,2100074692477486783,121.589964,39.097858,1,1580800229,20200204,train,1.0,4260,1580800058,2100074670098384550,1,121.569121,39.101864,121.573177,39.099671,631.0,11946,141,7926,1580801465,1580800506,1580799089,0.553453,0.488889,0.100000,1.1,0.170739,1407,448,959,-969
4,46490,107080018,4,2100074903132964329,121.520781,39.108080,0,1581253423,20200209,train,1.0,10446,1581252648,2100074904879007269,0,121.507304,39.116371,121.517294,39.090871,4063.0,33727,15069,3664,1581254349,1581252669,1581251863,0.547291,0.544444,0.433333,1.1,0.501748,1701,21,1680,-785


In [8]:
df_feature1 = df_feature1.loc[df_feature1['target'] == 1.0]
df_feature1['expect_diff'] = df_feature1['expect_time'] - df_feature1['current_time']
#df_feature1=df_feature1.drop(['id'],axis=1)
df_feature1.head()


,courier_id,wave_index,tracking_id,courier_wave_start_lng,courier_wave_start_lat,action_type,expect_time,date,type,target,group,id,current_time,last_tracking_id,last_action_type,source_lng,source_lat,target_lng,target_lat,grid_distance,TA,weather_grade,aoi_id,shop_id,promise_deliver_time,estimate_pick_time,assigned_time,speed,max_load,level,target_MHD,promise_dif,estimate_dif,Restime_dif,assign_dif,expect_diff
0,117052073,1,2100074527716017758,121.455805,39.108616,1,1580528357,20200201,train,1.0,692,2780,1580528211,2100074548913145283,1,121.441945,39.102244,121.439220,39.100178,532.0,0,5.0,1853,8358,1580530500,1580527009,1580527356,0.515605,0.655556,1.100000,0.154234,2289,-1202,3491,-855,146
2,117054417,0,2100074532411639390,121.448985,39.095307,1,1580525460,20200201,train,1.0,695,2796,1580525458,2100074546017437278,1,121.439220,39.100178,121.439220,39.100178,1.0,2,5.0,1130,8358,1580526900,1580472595,1580524836,0.664306,0.655556,1.100000,0.100000,1442,-52863,54305,-622,2
4,10401287,0,2100074533161764858,121.569268,39.104077,1,1580521211,20200201,train,1.0,279,673,1580521209,2100074542338410490,1,121.575545,39.103083,121.575545,39.103083,1.0,4,5.0,11454,4034,1580523300,1580516006,1580520725,0.547273,0.433333,1.100000,0.100000,2091,-5203,7294,-484,2
9,118662905,0,2100074534060164673,121.543779,39.081136,0,1580523911,20200201,train,1.0,772,3139,1580523617,2100074543608104054,0,121.547704,39.087431,121.547034,39.087641,137.0,9,5.0,2487,10592,1580525100,1580476524,1580523136,0.545046,0.600000,1.100000,0.109962,1483,-47093,48576,-481,294
14,10492242,0,2100074535358072827,121.495691,39.185221,0,1580522609,20200201,train,1.0,316,792,1580522095,2100074535358072827,1,121.515025,39.185017,121.519144,39.164451,2831.0,14,5.0,16981,684,1580523300,1580516414,1580520829,0.484566,0.600000,0.766667,0.379435,1205,-5681,6886,-1266,514


In [9]:
df_test = df_feature2[df_feature2['type'] == 'test'].copy()
df_train = df_feature1[df_feature1['type'] == 'train'].copy()


In [10]:
prediction = df_test[['courier_id', 'wave_index', 'tracking_id',
                      'courier_wave_start_lng', 'courier_wave_start_lat', 'action_type', 'expect_time', 'date']]
prediction['expect_time'] = 0

In [11]:
for f in df_feature1.select_dtypes('object'):
    if f not in ['date', 'type']:
        print(f)
        lbl = LabelEncoder()
        lbl = lbl.fit(df_train[f].astype(
            str).values.tolist()+df_test[f].astype(str).values.tolist())
        df_train[f] = lbl.transform(df_train[f].astype(str))
        df_test[f] = lbl.transform(df_test[f].astype(str))

In [42]:
ycol = 'expect_diff' #'promise_dif','Restime_dif','estimate_dif','target_MHD','grid_distance','speed','action_type','distance2_3'
feature_names = list(
    filter(lambda x: x not in [ycol,'assigned_time','TA', 'courier_id','expect_time', 'wave_index', 'tracking_id', 'target', 'date', 'type', 'group',
                               'courier_wave_start_lng', 'courier_wave_start_lat','group','current_time','last_tracking_id',
                              'last_action_type','source_lng','source_lat','target_lng','target_lat','aoi_id','shop_id',
                              'promise_deliver_time','estimate_pick_time','id'], df_train.columns))

model = lgb.LGBMRegressor(num_leaves=128,
                          max_depth=7,
                          learning_rate=0.05,
                          n_estimators=10000000,
                          subsample=0.8,
                          feature_fraction=0.8,
                          reg_alpha=0.5,
                          reg_lambda=0.5,
                          random_state=seed,
                          metric=None
                          )


oof = []
df_importance_list = []

kfold = GroupKFold(n_splits=5)
for fold_id, (trn_idx, val_idx) in enumerate(kfold.split(df_train[feature_names], df_train[ycol], df_train['group'])):
    X_train = df_train.iloc[trn_idx][feature_names]
    Y_train = df_train.iloc[trn_idx][ycol]

    X_val = df_train.iloc[val_idx][feature_names]
    Y_val = df_train.iloc[val_idx][ycol]

    print('\nFold_{} Training ================================\n'.format(fold_id+1))

    lgb_model = model.fit(X_train,
                          Y_train,
                          eval_names=['train', 'valid'],
                          eval_set=[(X_train, Y_train), (X_val, Y_val)],
                          verbose=500,
                          eval_metric='mae',
                          early_stopping_rounds=50)

    pred_val = lgb_model.predict(
        X_val, num_iteration=lgb_model.best_iteration_)
    df_oof = df_train.iloc[val_idx][['id', ycol]].copy()
    df_oof['pred'] = pred_val
    oof.append(df_oof)

    pred_test = lgb_model.predict(
        df_test[feature_names], num_iteration=lgb_model.best_iteration_)
    prediction['expect_time'] += (df_test['current_time'] + pred_test) / 5

    df_importance = pd.DataFrame({
        'column': feature_names,
        'importance': lgb_model.feature_importances_,
    })
    df_importance_list.append(df_importance)

    del lgb_model, pred_val, pred_test, X_train, Y_train, X_val, Y_val
    gc.collect()


Fold_1 Training ================================

Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[144]	train's l1: 130.793	valid's l1: 142.88

Fold_2 Training ================================

Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[204]	train's l1: 128.474	valid's l1: 141.307

Fold_3 Training ================================

Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[156]	train's l1: 130.398	valid's l1: 142.162

Fold_4 Training ================================

Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[150]	train's l1: 130.461	valid's l1: 144.174

Fold_5 Training ================================

Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[153]	train's l1: 131.165	valid's l1: 139.673


In [43]:
df_importance = pd.concat(df_importance_list)
df_importance = df_importance.groupby(['column'])['importance'].agg(
    'mean').sort_values(ascending=False).reset_index()
df_importance

,column,importance
0,promise_dif,1629.6
1,estimate_dif,1473.8
2,assign_dif,1313.6
3,Restime_dif,1177.8
4,speed,1171.2
5,grid_distance,1111.4
6,target_MHD,1089.0
7,max_load,497.0
8,action_type,351.8
9,weather_grade,208.8


In [44]:
df_oof = pd.concat(oof)
mae = metrics.mean_absolute_error(df_oof[ycol], df_oof['pred'])
print('mae:', mae)

mae: 142.03915240880937


In [41]:
prediction.loc[prediction.action_type==1,'action_type'] = 'PICKUP'
prediction.loc[prediction.action_type==0,'action_type'] = 'DELIVERY'

prediction.head(100)

,courier_id,wave_index,tracking_id,courier_wave_start_lng,courier_wave_start_lat,action_type,expect_time,date
44058,118009457,0,2100075783631734482,121.529745,39.277043,PICKUP,9.498688e+09,20200302
44059,115753441,1,2100075955398969002,121.536398,39.336073,PICKUP,9.498170e+09,20200301
44060,117054417,0,2100075987792069709,121.449907,39.095078,PICKUP,9.498679e+09,20200302
44061,106205842,7,2100076008354252272,121.403064,39.052707,DELIVERY,9.498370e+09,20200301
44062,117070713,1,2100076014436386205,121.666299,39.178807,PICKUP,9.498169e+09,20200301
44063,118808241,1,2100076016141305591,121.480258,39.073081,PICKUP,9.498180e+09,20200301
44064,10614152,1,2100076017499277056,121.494335,39.196944,PICKUP,9.498171e+09,20200301
44065,119535041,1,2100076017555244827,121.286610,38.942760,PICKUP,9.498212e+09,20200301
44066,110812290,1,2100076018369005246,121.528075,39.277048,DELIVERY,9.498185e+09,20200301
44067,118804441,1,2100076018644092811,121.517971,39.268645,DELIVERY,9.498170e+09,20200301


In [37]:
import zipfile
os.makedirs('./sub/{}'.format(int(mae)), exist_ok=True)
f = zipfile.ZipFile('./sub/{}.zip'.format(int(mae)), 'w', zipfile.ZIP_DEFLATED)
for date in prediction['date'].unique():
    df_temp = prediction[prediction['date'] == date]
    del df_temp['date']
    df_temp.to_csv('./sub/{}/action_{}.txt'.format(int(mae), date), index=False)
    f.write('./sub/{}/action_{}.txt'.format(int(mae), date), 'action_{}.txt'.format(date))
f.close()

In [22]:
'''
20200306 639
20200303 804
20200302 582
20200304 552
20200305 793
20200301 849
'''

'\n20200306 639\n20200303 804\n20200302 582\n20200304 552\n20200305 793\n20200301 849\n'